# StyleFusion-LoRA: Style-Mixing with LoRA Adapters on T5-Small

This notebook:
- Creates small style datasets (poetic, legal, journalistic)
- Fine-tunes separate LoRA adapters per style (only on query projections)
- Builds a mixed-style model by interpolating LoRA LoRA weights
- Evaluates style similarity and diversity
- Provides an interactive-style demo cell

We define a style-mixing model:




In [ ]:
!pip install -q transformers peft accelerate sentence-transformers datasets


In [ ]:
import os
import json
import random
from typing import List, Dict

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    get_linear_schedule_with_warmup
)

from peft import LoraConfig, get_peft_model, TaskType, PeftModel

from sentence_transformers import SentenceTransformer, util

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)



The verification routine performs the following functions:

1. **Existence Check:** Confirms that each required JSONL file is available in the dataset directory.
2. **Cardinality Assessment:** Reports the number of entries (lines) contained in each file to ensure sufficient training examples.
3. **Format Inspection:** Displays a sample line from each dataset to confirm adherence to the expected JSON Lines (JSONL) format, in which each entry is an independent JSON object.

This cell does not modify any data; it serves exclusively to ensure dataset completeness and correctness prior to model initialization and subsequent LoRA fine-tuning.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Base directory in Drive
BASE_DIR = "/content/drive/MyDrive/494finalProj/data"
os.makedirs(BASE_DIR, exist_ok=True)


In [ ]:
# Verify your existing JSONL files in Drive
import os

for style in ["poetic", "legal", "journalistic"]:
    path = os.path.join(BASE_DIR, f"{style}.jsonl")
    print(f"Checking: {path}")
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()
        print(f" - FOUND ({len(lines)} lines)")
        if lines:
            print(" - Example line:", lines[0].strip())
        else:
            print(" - The file is empty.")
    else:
        print(f" - MISSING! Please upload {style}.jsonl into {BASE_DIR}")

#Dataset Class Definition

This section defines the custom StyleDataset class, which serves as the data-loading component for the LoRA fine-tuning process. The class is responsible for:

Reading JSONL Files
Each style-specific dataset (poetic.jsonl, legal.jsonl, journalistic.jsonl) is stored in JSON Lines format, where every line is a standalone JSON object containing:

"style": the stylistic label

"input": the neutral prompt

"target": the style-specific rewritten completion

Tokenization & Preprocessing
The dataset class uses the T5 tokenizer to:

Convert each input and target string into token IDs

Apply padding and truncation

Prepare attention masks

Replace padding tokens in the target sequence with -100, ensuring they are ignored during loss computation

Returning Model-Ready Tensors
For each example, the class produces a dictionary containing:

1) input_ids

2) attention_mask

3) labels

These outputs follow HuggingFace’s expected format for conditional sequence-to-sequence training and will be used directly by the T5 model with LoRA adapters.
This dataset abstraction enables clean, modular training for each individual writing style.




In [ ]:
class StyleDataset(Dataset):
    """Enhanced dataset with stronger T5 task formatting"""

    def __init__(self, path, tokenizer, max_length=128):
        self.samples = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                data = json.loads(line)

                style = data.get("style", "")
                input_text = data["input"]

                # STRONGER prompt engineering for T5
                # Use multiple strategies to force English and style
                input_text = f"rewrite in {style} English style: {input_text}"

                data["formatted_input"] = input_text
                self.samples.append(data)

        self.tokenizer = tokenizer
        self.max_length = max_length
        print(f"Loaded {len(self.samples)} samples")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ex = self.samples[idx]

        input_text = ex["formatted_input"]
        target_text = ex["target"]

        # Tokenize input
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize target
        labels = self.tokenizer(
            text_target=target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"]

        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        labels = labels.squeeze(0)

        # Mask padding tokens
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

#Loading the Base T5 Model and Tokenizer

This cell initializes the `t5-small` model and its corresponding tokenizer from the
HuggingFace Transformers library. The tokenizer handles text preprocessing, including
token-to-ID conversion, padding, and truncation. The model itself serves as the
frozen backbone for all subsequent LoRA-based fine-tuning.

No parameters of the base T5 network are modified at this stage; it provides a
consistent, shared starting point for all style-specific LoRA adapters trained later
in the notebook. By loading the model onto the available device (GPU if present),
we ensure efficient training performance in downstream cells.


In [ ]:
BASE_MODEL_NAME = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
base_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
base_model = base_model.to(device)

print("Base model loaded:", BASE_MODEL_NAME)


# LoRA Configuration

This cell defines a helper function that attaches a Low-Rank Adaptation (LoRA)
module to the base T5 model. The configuration targets only the *query projection*
(`Wq`) parameters within the transformer attention layers.

By restricting trainable parameters to this specific subcomponent, the fine-tuning
process becomes more parameter-efficient while still enabling strong stylistic
adaptation. This modular design also allows each style (poetic, legal, journalistic)
to maintain its own LoRA adapter, enabling later interpolation for style blending.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

def add_lora_to_model(model):
    """
    More aggressive LoRA targeting:
    - Target both q and v (query and value)
    - Higher rank for more capacity
    - Target more modules for stronger adaptation
    """
    lora_config = LoraConfig(
        r=32,  # Increased from 16
        lora_alpha=64,  # Increased from 32
        target_modules=["q", "v", "k", "o"],  # Target all attention matrices
        lora_dropout=0.05,  # Lower dropout
        bias="none",
        task_type=TaskType.SEQ_2_SEQ_LM,
    )
    peft_model = get_peft_model(model, lora_config)

    # Print trainable parameters
    trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in peft_model.parameters())
    print(f"Trainable: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.2f}%)")

    return peft_model


In [ ]:
def train_style_lora(
    style: str,
    num_epochs: int = 100,  # MUCH higher - you need at least 50-100 epochs
    batch_size: int = 2,    # Keep small for your dataset size
    lr: float = 5e-4,       # Try slightly lower learning rate
    max_length: int = 128,
    warmup_ratio: float = 0.1
):
    """Aggressive training to overcome T5's multilingual nature"""

    print(f"\n{'='*60}")
    print(f"Training style: {style}")
    print(f"{'='*60}")

    data_path = os.path.join(DATA_DIR, f"{style}.jsonl")
    ckpt_path = os.path.join(CKPT_DIR, style)
    os.makedirs(ckpt_path, exist_ok=True)

    # Fresh model
    model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
    model = model.to(device)
    model = add_lora_to_model(model)

    # Freeze base
    for name, param in model.named_parameters():
        if "lora_" not in name:
            param.requires_grad = False

    # Load data
    train_dataset = StyleDataset(data_path, tokenizer, max_length=max_length)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Optimizer with higher LR
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr,
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )

    # Scheduler
    total_steps = num_epochs * len(train_loader)
    warmup_steps = int(total_steps * warmup_ratio)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    print(f"Total steps: {total_steps}, Warmup: {warmup_steps}")

    model.train()
    best_loss = float('inf')
    patience = 20  # Increased patience
    no_improve = 0

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch in train_loader:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)

        # Print every 10 epochs or when improved
        if (epoch + 1) % 10 == 0 or avg_loss < best_loss:
            print(f"Epoch {epoch+1:3d}/{num_epochs} - Loss: {avg_loss:.4f}", end="")

        # Save best
        if avg_loss < best_loss:
            improvement = best_loss - avg_loss if best_loss != float('inf') else 0
            best_loss = avg_loss
            no_improve = 0
            model.save_pretrained(ckpt_path)
            if (epoch + 1) % 10 == 0 or avg_loss < best_loss:
                print(f" ✓ SAVED (↓{improvement:.4f})")
        else:
            if (epoch + 1) % 10 == 0:
                print()
            no_improve += 1

        # Early stopping - but not too early
        if no_improve >= patience and epoch > 50:
            print(f"\nEarly stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
            break

    print(f"\nBest loss: {best_loss:.4f}")
    print(f"Model saved to: {ckpt_path}\n")

    return model

# Train all Styles

## Style-Specific Model Loading and Text Generation

This cell provides two utility functions essential for evaluating the behavior of
each independently trained LoRA adapter.

**1. `load_style_model(style)`**  
This function reconstructs a style-specific model by:
- Loading a fresh instance of the base `t5-small` model  
- Attaching the preconfigured LoRA architecture to expose the trainable query-projection matrices  
- Importing the corresponding LoRA adapter weights saved during training  
- Moving the resulting composite model to the appropriate device (GPU or CPU)  
- Setting the model to evaluation mode to ensure deterministic inference  

This procedure enables modular loading of each stylistic adapter without retraining.

**2. `generate_text(model, prompt)`**  
This function performs inference by:
- Tokenizing the input prompt  
- Using nucleus sampling (`top_p`) with temperature scaling to promote stylistic expression  
- Generating a continuation conditioned on the given LoRA-modified model  
- Decoding the output tokens into a human-readable string  

Together, these functions enable qualitative assessment of the learned styles,
allowing the researcher to compare the outputs of the poetic, legal, and
journalistic adapters in isolation before conducting style-mixing experiments.


In [ ]:
STYLES = ["poetic", "legal", "journalistic"]
DATA_DIR = BASE_DIR
CKPT_DIR = os.path.join(BASE_DIR, "checkpoints")
os.makedirs(CKPT_DIR, exist_ok=True)

for style in STYLES:
    train_style_lora(
        style,
        num_epochs=30,  # Much more epochs
        batch_size=4,    # Small batch for small dataset
        lr=5e-4          # Adjusted learning rate
    )


Training style: poetic
Trainable: 2,359,296 / 62,865,920 (3.75%)
Loaded 20 samples
Total steps: 150, Warmup: 15
Epoch   1/30 - Loss: 5.6274Epoch   2/30 - Loss: 5.5939Epoch   3/30 - Loss: 5.2401Epoch   4/30 - Loss: 4.7377Epoch   5/30 - Loss: 4.4036Epoch   6/30 - Loss: 4.3226Epoch   7/30 - Loss: 4.1850Epoch   8/30 - Loss: 3.9488Epoch   9/30 - Loss: 3.8806Epoch  10/30 - Loss: 3.7993 ✓ SAVED (↓0.0813)
Epoch  11/30 - Loss: 3.7200Epoch  12/30 - Loss: 3.6058Epoch  13/30 - Loss: 3.5343Epoch  14/30 - Loss: 3.5031Epoch  15/30 - Loss: 3.4464Epoch  16/30 - Loss: 3.3771Epoch  17/30 - Loss: 3.3633Epoch  18/30 - Loss: 3.3194Epoch  19/30 - Loss: 3.2397Epoch  20/30 - Loss: 3.1596 ✓ SAVED (↓0.0801)
Epoch  22/30 - Loss: 3.1156Epoch  24/30 - Loss: 3.1087Epoch  25/30 - Loss: 3.1067Epoch  26/30 - Loss: 3.0177Epoch  30/30 - Loss: 3.0210

Best loss: 3.0177
Model saved to: /content/drive/MyDrive/494finalProj/data/checkpoints/poetic


Training style: legal
Trainable: 2,359,296 / 62,865,920 (3.75%)
Loaded 20 sa

In [ ]:
def load_style_model(style: str):
    ckpt_path = os.path.join(CKPT_DIR, style)
    base = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
    base = base.to(device)
    peft_model = PeftModel.from_pretrained(base, ckpt_path)
    peft_model = peft_model.to(device)
    peft_model.eval()
    return peft_model


def generate_text(model, prompt: str, max_new_tokens: int = 64):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # More deterministic for testing
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        min_new_tokens=10,  # Force at least some output
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.2,  # Discourage repetition
        num_beams=1,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
test_prompt_base = "Describe the night sky above a courthouse."

for style in STYLES:
    # Test with simple prompt (no style prefix)
    m = load_style_model(style)
    out = generate_text(m, test_prompt_base)
    print("=" * 60)
    print("STYLE:", style)
    print("PROMPT:", test_prompt_base)
    print("OUTPUT:", out)




##Build a Mixed-Style Model by Interpolating LoRA Weights

We will create a mixed model by linearly combining the LoRA parameters from each style by α weights.

In [ ]:
from typing import List
from transformers import AutoModelForSeq2SeqLM

# 4. Improved mixed model loading with better error handling
def build_mixed_lora_model(styles: List[str], alphas: List[float]):
    assert len(styles) == len(alphas), "styles and alphas length mismatch"
    s = sum(alphas)
    if abs(s - 1.0) > 1e-6:
        alphas = [a / s for a in alphas]

    # Start from fresh base
    base = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
    base = base.to(device)

    # Load first adapter as base
    first_ckpt = os.path.join(CKPT_DIR, styles[0])
    mixed = PeftModel.from_pretrained(base, first_ckpt)
    mixed_sd = mixed.state_dict()

    # Collect all adapter state dicts
    style_sds = []
    for style in styles:
        ckpt_path = os.path.join(CKPT_DIR, style)
        temp_base = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL_NAME)
        temp_peft = PeftModel.from_pretrained(temp_base, ckpt_path)
        style_sds.append(temp_peft.state_dict())
        del temp_base, temp_peft  # Free memory

    # Interpolate only LoRA parameters
    for key in mixed_sd.keys():
        if "lora_A" in key or "lora_B" in key:
            weighted_param = None
            for alpha, sd in zip(alphas, style_sds):
                if key in sd:
                    if weighted_param is None:
                        weighted_param = alpha * sd[key].clone()
                    else:
                        weighted_param += alpha * sd[key]

            if weighted_param is not None:
                mixed_sd[key] = weighted_param

    mixed.load_state_dict(mixed_sd, strict=False)  # Allow missing keys
    mixed.eval()
    return mixed

def generate_mixed(prompt: str, styles: List[str], alphas: List[float], max_new_tokens: int = 64):
    model = build_mixed_lora_model(styles, alphas)
    return generate_text(model, prompt, max_new_tokens=max_new_tokens)

Test thw mixx

In [ ]:
BASE_MODEL_NAME = "t5-small"

prompt = "Describe a contract breach in a dramatic way."
styles_mix = ["poetic", "legal"]
alphas_mix = [0.5, 0.5]

output = generate_mixed(prompt, styles_mix, alphas_mix)
print("PROMPT:", prompt)
print("STYLES:", styles_mix)
print("ALPHAS:", alphas_mix)
print("OUTPUT:", output)


##Embedding Model & Style Centers

Now we evaluate style similarity by Load SentenceTransformer ("all-MiniLM-L6-v2") to evaluate style characteristics.

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2").to(device)
print("Embedding model loaded.")

def compute_style_center(style: str, num_samples: int = 8):
    path = os.path.join(DATA_DIR, f"{style}.jsonl")
    texts = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            ex = json.loads(line)
            texts.append(ex["target"])
    random.shuffle(texts)
    texts = texts[:num_samples]
    embeddings = embed_model.encode(texts, convert_to_tensor=True)
    center = embeddings.mean(dim=0)
    center = center / center.norm()
    return center

style_centers = {style: compute_style_center(style) for style in STYLES}
print("Computed style centers for:", style_centers.keys())


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.
Computed style centers for: dict_keys(['poetic', 'legal', 'journalistic'])


###Similarity Evaluation for Pure & Mixed Outputs

In [ ]:
eval_prompts = [
    "Describe the night sky above a courthouse.",
    "Explain the importance of justice.",
    "Describe a storm during a legal trial."
]

def style_similarity(text: str):
    emb = embed_model.encode(text, convert_to_tensor=True)
    emb = emb / emb.norm()
    sims = {}
    for style, center in style_centers.items():
        sims[style] = float(util.cos_sim(emb, center).item())
    return sims

results = []

for prompt in eval_prompts:
    # pure styles
    for style in STYLES:
        m = load_style_model(style)
        out = generate_text(m, prompt)
        sims = style_similarity(out)
        results.append({
            "prompt": prompt,
            "config": f"pure-{style}",
            "output": out,
            **{f"sim_{s}": v for s, v in sims.items()}
        })
    # one mixed example: poetic + legal
    mix_styles = ["poetic", "legal"]
    mix_alphas = [0.5, 0.5]
    out_mix = generate_mixed(prompt, mix_styles, mix_alphas)
    sims_mix = style_similarity(out_mix)
    results.append({
        "prompt": prompt,
        "config": "mix-poetic-legal-0.5-0.5",
        "output": out_mix,
        **{f"sim_{s}": v for s, v in sims_mix.items()}
    })

print("Number of eval rows:", len(results))

Number of eval rows: 12


##View Similarity Table

In [ ]:
import pandas as pd

df_results = pd.DataFrame(results)
display(df_results[["prompt", "config", "sim_poetic", "sim_legal", "sim_journalistic"]])


,prompt,config,sim_poetic,sim_legal,sim_journalistic
0,Describe the night sky above a courthouse.,pure-poetic,0.490308,0.235823,0.396609
1,Describe the night sky above a courthouse.,pure-legal,0.376738,0.342210,0.424189
2,Describe the night sky above a courthouse.,pure-journalistic,0.312644,0.187728,0.304772
3,Describe the night sky above a courthouse.,mix-poetic-legal-0.5-0.5,0.245756,0.286120,0.265162
4,Explain the importance of justice.,pure-poetic,0.260960,0.313505,0.117280
5,Explain the importance of justice.,pure-legal,0.095827,0.550881,0.112801
6,Explain the importance of justice.,pure-journalistic,0.226329,0.161971,0.202740
7,Explain the importance of justice.,mix-poetic-legal-0.5-0.5,0.040572,0.396203,0.161965
8,Describe a storm during a legal trial.,pure-poetic,0.515152,0.020236,0.358221
9,Describe a storm during a legal trial.,pure-legal,0.314339,0.463645,0.342330


In [ ]:
display(df_results[["prompt", "config", "output"]])


,prompt,config,output
0,Describe the night sky above a courthouse.,pure-poetic,The courthouse walked across the sky and saw t...
1,Describe the night sky above a courthouse.,pure-legal,"Upon viewing the night sky, the courthouse is ..."
2,Describe the night sky above a courthouse.,pure-journalistic,"Sky above a courthouse stands at the skyline, ..."
3,Describe the night sky above a courthouse.,mix-poetic-legal-0.5-0.5,Veuillez décrire le sky nocturne de la faune a...
4,Explain the importance of justice.,pure-poetic,Justice is the right thing. Read the story wit...
5,Explain the importance of justice.,pure-legal,"Justice is a matter of fact, without any prior..."
6,Explain the importance of justice.,pure-journalistic,The right message was given to the nation's y...
7,Explain the importance of justice.,mix-poetic-legal-0.5-0.5,Les arguments qui suggèrent que justice est im...
8,Describe a storm during a legal trial.,pure-poetic,"The storm poured through the streets, all the ..."
9,Describe a storm during a legal trial.,pure-legal,Storms are a legal proceeding involving a cour...


##Distinct-n Diversity Metrics
Compute:


*   distinct - 1
*   distinct - 2


Evaluate lexical diversity for each style and mixed model configuration.

In [ ]:
import pandas as pd

def distinct_n(texts: List[str], n: int = 1) -> float:
    ngrams = set()
    total = 0
    for t in texts:
        tokens = t.split()
        if len(tokens) < n:
            continue
        total += len(tokens) - n + 1
        for i in range(len(tokens) - n + 1):
            ngrams.add(tuple(tokens[i:i+n]))
    return len(ngrams) / total if total > 0 else 0.0

# Re-create df_results here since it was not defined in the current execution flow
df_results = pd.DataFrame(results)

div_rows = []
for config in df_results["config"].unique():
    subset = df_results[df_results["config"] == config]
    texts = subset["output"].tolist()
    d1 = distinct_n(texts, n=1)
    d2 = distinct_n(texts, n=2)
    div_rows.append({"config": config, "distinct1": d1, "distinct2": d2})

df_div = pd.DataFrame(div_rows)
display(df_div)

,config,distinct1,distinct2
0,pure-poetic,0.771429,0.968750
1,pure-legal,0.777778,1.000000
2,pure-journalistic,0.829268,0.973684
3,mix-poetic-legal-0.5-0.5,1.000000,1.000000


from matplotlib import pyplot as plt
df_div['distinct1'].plot(kind='hist', bins=20, title='distinct1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_div['distinct2'].plot(kind='hist', bins=20, title='distinct2')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df_div.groupby('config').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_div.plot(kind='scatter', x='distinct1', y='distinct2', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df_div['distinct2'].plot(kind='line', figsize=(8, 4), title='distinct2')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_div['config'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_div, x='distinct1', y='config', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df_div['config'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df_div, x='distinct2', y='config', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

##Simple Demo Cell (Interactive-ish)

In [ ]:
prompt = "Describe a peaceful morning in a court building."
styles_demo = ["poetic", "legal", "journalistic"]
alphas_demo = [0.5, 0.3, 0.2]  # will be normalized automatically

print("PROMPT:", prompt)
print("STYLES:", styles_demo)
print("ALPHAS:", alphas_demo)
out = generate_mixed(prompt, styles_demo, alphas_demo)
print("\nOUTPUT:")
print(out)

print("\nStyle similarities:")
print(style_similarity(out))


PROMPT: Describe a peaceful morning in a court building.
STYLES: ['poetic', 'legal', 'journalistic']
ALPHAS: [0.5, 0.3, 0.2]

OUTPUT:
A peaceful morning in a court building is a quiet morning.

Style similarities:
{'poetic': 0.29994094371795654, 'legal': 0.29799559712409973, 'journalistic': 0.5222954750061035}


Export Results & Notes

In [ ]:
RESULTS_DIR = os.path.join(BASE_DIR, "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

sim_path = os.path.join(RESULTS_DIR, "similarity_results.csv")
div_path = os.path.join(RESULTS_DIR, "diversity_results.csv")

df_results.to_csv(sim_path, index=False)
df_div.to_csv(div_path, index=False)

print("Saved similarity results to:", sim_path)
print("Saved diversity results to:", div_path)